In [1]:
%matplotlib inline
import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

import pickle
import pandas as pd
import csv
import os
import time
import datetime
from tensorflow.contrib import learn
from nltk.tokenize import TweetTokenizer

In [2]:
def load_data_and_labels(train_data_file, test_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    train_data =  pd.read_csv(train_data_file, sep='\t', quoting=csv.QUOTE_NONE, header=None, names=['id', 'language', 'label', 'text'])
    test_data = pd.read_csv(test_data_file, sep='\t', quoting=csv.QUOTE_NONE, header=None, names=['id', 'language', 'label', 'text'])
    
    x_train = train_data['text'].tolist()
    y_train = train_data['label'].tolist()

    x_test = train_data['text'].tolist()
    y_test = train_data['label'].tolist()
    
    x_train = [s.strip() for s in x_train]
    x_test = [s.strip() for s in x_test]
    
    y_train_encoding = []
    for label in y_train:
        if label=='neutral':
            y_train_encoding.append([0, 1, 0])
        elif label=='positive':
            y_train_encoding.append([0, 0, 1])
        elif label=='negative':
            y_train_encoding.append([1, 0, 0])
    
    y_test_encoding = []
    for label in y_test:
        if label=='neutral':
            y_train_encoding.append([0, 1, 0])
        elif label=='positive':
            y_train_encoding.append([0, 0, 1])
        elif label=='negative':
            y_train_encoding.append([1, 0, 0])
    
    return [x_train, y_train_encoding, x_test, y_test_encoding]

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [3]:
final_embeddings = np.load('./data/embed_tweets_en_200M_200D/embedding_matrix.npy')
word_dict = {}
with open('./data/embed_tweets_en_200M_200D/vocabulary.pickle', 'rb') as myfile:
    word_dict = pickle.load(myfile)

In [4]:
#Load label data
x_train_sentence, y_train, x_test_sentence, y_test = load_data_and_labels('./data/supervised_data/en_train.tsv', './data/supervised_data/en_test.tsv')

max_document_length = max([len(x.split(" ")) for x in x_train_sentence])


In [5]:
tknzr = TweetTokenizer()
x_train = []
for tweet in x_train_sentence:
    temp = np.zeros(max_document_length, dtype=np.int).tolist()
    for index, word in enumerate(tknzr.tokenize(tweet)):
        word = word.lower()
        if word in word_dict:
            temp[index] = word_dict[word][0]
    x_train.append(temp)
x_train = np.array(x_train)

In [10]:
# put Word2Vec on 590 million English Tweets using 52 dimensions.
vocabulary_size = 1859185
embedding_size = 200  # Dimension of the embedding vector.
graph = tf.Graph()

sequence_length=x_train.shape[1]
num_classes=3

# filter_sizes: The number of words we want our convolutional filters to cover. 
# We will have num_filters for each size specified here. 
# For example, [3, 4, 5] means that we will have filters that slide over 3, 4 and 5 words respectively, for a total of 3 * num_filters filters.
filter_sizes = [4]

first_filter_window_size = 4
second_filter_window_size = 3
num_filters = 200

l2_reg_lambda=0.0

with graph.as_default():
    input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
    input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
    
    embeddings = tf.Variable(tf.constant(0.0, shape=[vocabulary_size, embedding_size]),
                    trainable=False, name="W")

    embedding_placeholder = tf.placeholder(tf.float32, [vocabulary_size, embedding_size])
    embedding_init = embeddings.assign(embedding_placeholder)  # assign exist word embeddings

    embedded_chars = tf.nn.embedding_lookup(embeddings, input_x)
    embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)
    print(input_x.shape)
    print(embedded_chars_expanded.shape)
    # Keeping track of l2 regularization loss (optional)
    l2_loss = tf.constant(0.0)
    
     # Add variable initializer.
    init = tf.global_variables_initializer()
    
    
    # Create first cnn : a convolution + maxpool layer for each filter size    
    # Convolution Layer
    pooled_outputs = []
    for i, filter_size in enumerate(filter_sizes):
        with tf.name_scope("conv-maxpool-%s" % filter_size):
            # Convolution Layer
            filter_shape = [first_filter_window_size, embedding_size, 1, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            print("CNN filter", W.shape)
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(
                embedded_chars_expanded,
                W,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            # Apply nonlinearity
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            # Maxpooling over the outputs
            pooled = tf.nn.max_pool(
                h,
                ksize=[1, sequence_length - filter_size + 1, 1, 1],
                strides=[1, 2, 2, 1],
                padding='VALID',
                name="pool")
            pooled_outputs.append(pooled)
    print("conv", conv.shape)

    
#     filter_shape = [first_filter_window_size, embedding_size, 1, num_filters]
#     filter_shape = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
#     print("filter_shape", filter_shape.shape)
#     b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
    
#     conv = tf.nn.conv2d(
#         embedded_chars_expanded,
#         filter_shape,
#         strides=[1, 1, 1, 1],  # strides size change will cause cnn result different
#         padding="VALID",
#         name="conv")
    


#     # Apply nonlinearity
#     h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
#     # Maxpooling over the outputs
#     pooled = tf.nn.max_pool(
#         h,
#         ksize=[1, sequence_length - first_filter_window_size + 1, 1, 1],  
#         strides=[1, 2, 2, 1],
#         padding='VALID',
#         name="pool")
    
    print("pooled", pooled.shape)
    
    

    
#     # Create second cnn : a convolution + maxpool layer for each filter size
#     pooled_outputs = []
#     # Convolution Layer
#     filter_shape = [second_filter_window_size, embedding_size, 1, num_filters]
#     filter_shape = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
#     b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
#     conv = tf.nn.conv2d(
#         self.embedded_chars_expanded,
#         filter_shape,
#         strides=[1, 1, 1, 1],  # strides size change will cause cnn result different
#         padding="VALID",
#         name="conv")
#     # Apply nonlinearity
#     h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
#     # Maxpooling over the outputs
#     pooled = tf.nn.max_pool(
#         h,
#         ksize=[1, sequence_length - second_filter_window_size + 1, 1, 1],  
#         strides=[1, 1, 1, 1],
#         padding='VALID',
#         name="pool")
#     pooled_outputs.append(pooled)
            
            
            
            
     # Combine all the pooled features
#     num_filters_total = num_filters * len(filter_sizes)
    h_pool_flat = tf.reshape(pooled, [-1, num_filters])  # flatten pooling layers
    print("h_pool_flat", h_pool_flat.shape)
    
    # Add dropout
#     with tf.name_scope("dropout"):
#         self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

    # Final (unnormalized) scores and predictions
    with tf.name_scope("output"):
        W = tf.get_variable(
            "W",
            shape=[num_filters, num_classes],
            initializer=tf.contrib.layers.xavier_initializer())
        b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
        l2_loss += tf.nn.l2_loss(W)
        l2_loss += tf.nn.l2_loss(b)
        scores = tf.nn.xw_plus_b(h_pool_flat, W, b, name="scores")
        print("scores", scores.shape)
        predictions = tf.argmax(scores, 1, name="predictions")
        print("predictions", predictions.shape)


    # Calculate mean cross-entropy loss
    with tf.name_scope("loss"):
        losses = tf.nn.softmax_cross_entropy_with_logits(logits=scores, labels=input_y)
        print("losses", losses.shape)
        loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

    # Accuracy
    with tf.name_scope("accuracy"):
        correct_predictions = tf.equal(predictions, tf.argmax(input_y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
    


(?, 44)
(?, 44, 200, 1)
CNN filter (4, 200, 1, 200)
conv (?, 41, 1, 200)
pooled (?, 1, 1, 200)
h_pool_flat (?, 200)
scores (?, 3)
predictions (?,)
losses (?,)


In [11]:
batch_size = 32
num_checkpoints = 5
num_epochs = 200
evaluate_every = 100
allow_soft_placement=True
log_device_placement=False

with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=allow_soft_placement,
      log_device_placement=log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        
#         # Define Training procedure
#         global_step = tf.Variable(0, name="global_step", trainable=False)
#         optimizer = tf.train.AdamOptimizer(1e-3)
#         grads_and_vars = optimizer.compute_gradients(loss)
#         train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

#         # Keep track of gradient values and sparsity (optional)
#         grad_summaries = []
#         for g, v in grads_and_vars:
#             if g is not None:
#                 grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
#                 sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
#                 grad_summaries.append(grad_hist_summary)
#                 grad_summaries.append(sparsity_summary)
#         grad_summaries_merged = tf.summary.merge(grad_summaries)

#         # Output directory for models and summaries
#         timestamp = str(int(time.time()))
#         out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
#         print("Writing to {}\n".format(out_dir))

#         # Summaries for loss and accuracy
#         loss_summary = tf.summary.scalar("loss", loss)
#         acc_summary = tf.summary.scalar("accuracy", accuracy)

#         # Train Summaries
#         train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
#         train_summary_dir = os.path.join(out_dir, "summaries", "train")
#         train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

#         # Dev summaries
#         dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
#         dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
#         dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

#         # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
#         checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
#         checkpoint_prefix = os.path.join(checkpoint_dir, "model")
#         if not os.path.exists(checkpoint_dir):
#             os.makedirs(checkpoint_dir)
#         saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

#         # Write vocabulary
#         vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())


        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              input_x: x_batch,
              input_y: y_batch,
            }
#             _, step, summaries, loss, accuracy = sess.run(
#                 [train_op, global_step, train_summary_op, loss, accuracy],
#                 feed_dict)
#             time_str = datetime.datetime.now().isoformat()
#             print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
#             train_summary_writer.add_summary(summaries, step)

            loss_log, accuracy_log = sess.run([loss, accuracy], feed_dict)
            print(loss_log, accuracy_log)

#         def dev_step(x_batch, y_batch, writer=None):
#             """
#             Evaluates model on a dev set
#             """
#             feed_dict = {
#               input_x: x_batch,
#               input_y: y_batch,
#             }
#             step, summaries, loss, accuracy = sess.run(
#                 [global_step, dev_summary_op, loss, accuracy],
#                 feed_dict)
#             time_str = datetime.datetime.now().isoformat()
#             print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
#             if writer:
#                 writer.add_summary(summaries, step)

        
        sess.run(embedding_init, feed_dict={embedding_placeholder: final_embeddings})
        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), batch_size, num_epochs)
        
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            
#             current_step = tf.train.global_step(sess, global_step)
#             if current_step % evaluate_every == 0:
#                 print("\nEvaluation:")
#                 dev_step(x_dev, y_dev, writer=dev_summary_writer)
#                 print("")
#             if current_step % checkpoint_every == 0:
#                 path = saver.save(sess, checkpoint_prefix, global_step=current_step)
#                 print("Saved model checkpoint to {}\n".format(path))

1.54729 0.28125
1.50138 0.34375
1.57471 0.21875
1.36212 0.34375
1.37131 0.34375
1.33426 0.3125
1.47944 0.375
1.29609 0.40625
1.26986 0.34375
1.56581 0.1875
1.40304 0.34375
1.52512 0.21875
1.48043 0.21875
1.41746 0.28125
1.49064 0.28125
1.50932 0.375
1.64617 0.15625
1.59021 0.1875
1.55211 0.3125
1.43321 0.3125
1.41053 0.28125
1.35605 0.40625
1.22181 0.375
1.34949 0.34375
1.42339 0.34375
1.34702 0.34375
1.3155 0.5
1.59681 0.1875
1.38235 0.375
1.26542 0.375
1.54375 0.21875
1.69492 0.1875
1.34433 0.375
1.35922 0.28125
1.41782 0.3125
1.45142 0.375
1.51443 0.25
1.57707 0.25
1.30897 0.4375
1.40422 0.375
1.66725 0.15625
1.43676 0.28125
1.32773 0.3125
1.33471 0.375
1.58397 0.1875
1.52564 0.1875
1.48692 0.34375
1.57594 0.1875
1.50116 0.28125
1.44337 0.28125
1.5989 0.1875
1.45104 0.34375
1.46942 0.25
1.53098 0.21875
1.22644 0.375
1.42282 0.34375
1.58719 0.21875
1.24817 0.5
1.4195 0.40625
1.43746 0.40625
1.51751 0.28125
1.27858 0.40625
1.68455 0.25
1.78736 0.125
1.30596 0.4375
1.55649 0.15625
1.24

1.54923 0.1875
1.30209 0.28125
1.46953 0.3125
1.42575 0.25
1.34198 0.28125
1.65361 0.25
1.53148 0.3125
1.55221 0.1875
1.30592 0.375
1.26916 0.375
1.17228 0.5
1.24526 0.46875
1.33932 0.34375
1.33343 0.34375
1.56978 0.21875
1.28211 0.5
1.44306 0.34375
1.47741 0.34375
1.5548 0.25
1.59596 0.25
1.60389 0.1875
1.41989 0.34375
1.32768 0.34375
1.75738 0.15625
1.56774 0.25
1.42318 0.25
1.43088 0.34375
1.51844 0.25
1.5301 0.21875
1.54334 0.34375
1.45631 0.25
1.59439 0.28125
1.42085 0.375
1.371 0.34375
1.59363 0.21875
1.4812 0.34375
1.63657 0.21875
1.55602 0.25
1.61118 0.21875
1.6288 0.3125
1.62886 0.28125
1.5633 0.1875
1.43349 0.34375
1.32025 0.40625
1.43723 0.375
1.47789 0.25
1.57684 0.21875
1.60436 0.3125
1.28945 0.34375
1.52737 0.21875
1.44549 0.21875
1.33132 0.34375
1.29788 0.4375
1.36865 0.375
1.58174 0.21875
1.41706 0.25
1.37236 0.28125
1.43145 0.34375
1.54176 0.1875
1.42103 0.25
1.56498 0.3125
1.37589 0.375
1.4622 0.3125
1.34234 0.40625
1.12764 0.53125
1.41478 0.34375
1.73649 0.15625
1.52

1.41985 0.25
1.6449 0.21875
1.58581 0.21875
1.41093 0.4375
1.48556 0.34375
1.45837 0.3125
1.54155 0.25
1.53706 0.3125
1.35746 0.392857
1.42374 0.40625
1.61 0.3125
1.61142 0.1875
1.48796 0.375
1.58988 0.3125
1.54216 0.25
1.34255 0.40625
1.46272 0.3125
1.34599 0.40625
1.62477 0.15625
1.44363 0.34375
1.3651 0.25
1.50396 0.28125
1.52753 0.25
1.45499 0.28125
1.44485 0.28125
1.38214 0.34375
1.4607 0.375
1.40975 0.3125
1.53155 0.21875
1.51607 0.25
1.58337 0.28125
1.76965 0.15625
1.32735 0.21875
1.61562 0.28125
1.43065 0.40625
1.59353 0.25
1.68892 0.125
1.42773 0.3125
1.39773 0.4375
1.44884 0.34375
1.51977 0.25
1.45282 0.25
1.64607 0.25
1.2661 0.5
1.50711 0.25
1.59977 0.28125
1.66203 0.15625
1.35098 0.3125
1.53223 0.28125
1.37215 0.40625
1.54509 0.34375
1.53833 0.25
1.4802 0.25
1.63087 0.1875
1.41089 0.375
1.50697 0.1875
1.26105 0.5
1.5017 0.3125
1.60231 0.1875
1.68368 0.15625
1.42705 0.3125
1.55018 0.21875
1.68971 0.1875
1.39141 0.375
1.46759 0.375
1.34356 0.40625
1.44986 0.28125
1.47207 0.31

1.44933 0.34375
1.50319 0.25
1.25476 0.4375
1.39661 0.25
1.46606 0.34375
1.55861 0.25
1.59905 0.21875
1.54902 0.25
1.5273 0.34375
1.39075 0.28125
1.20128 0.40625
1.64766 0.1875
1.36175 0.357143
1.27562 0.34375
1.45854 0.34375
1.28048 0.375
1.56403 0.1875
1.3162 0.4375
1.56847 0.21875
1.46296 0.40625
1.5003 0.25
1.71791 0.125
1.38886 0.25
1.53549 0.25
1.53469 0.25
1.49966 0.25
1.56353 0.3125
1.44582 0.25
1.47218 0.28125
1.36895 0.375
1.30154 0.3125
1.55067 0.21875
1.4231 0.40625
1.35087 0.3125
1.64379 0.21875
1.53983 0.3125
1.60613 0.125
1.34401 0.3125
1.35354 0.40625
1.42203 0.25
1.55071 0.21875
1.63177 0.28125
1.44009 0.34375
1.64673 0.1875
1.58747 0.25
1.33499 0.375
1.34741 0.46875
1.47696 0.125
1.49645 0.28125
1.31042 0.40625
1.23494 0.4375
1.60999 0.1875
1.87467 0.125
1.53239 0.28125
1.28016 0.40625
1.40875 0.375
1.48867 0.34375
1.63908 0.25
1.40346 0.3125
1.33695 0.40625
1.43292 0.34375
1.58367 0.25
1.52186 0.28125
1.35522 0.40625
1.55737 0.21875
1.38834 0.34375
1.69058 0.1875
1.6

1.44541 0.28125
1.46672 0.3125
1.47397 0.3125
1.62126 0.125
1.50996 0.34375
1.53803 0.28125
1.61343 0.25
1.66342 0.25
1.40438 0.3125
1.30799 0.4375
1.42627 0.375
1.81704 0.15625
1.40002 0.34375
1.58101 0.28125
1.59941 0.21875
1.32148 0.4375
1.26141 0.40625
1.60572 0.15625
1.5042 0.3125
1.50745 0.21875
1.53825 0.25
1.37798 0.21875
1.31098 0.34375
1.67046 0.1875
1.48272 0.34375
1.39282 0.34375
1.45539 0.25
1.34717 0.285714
1.61949 0.15625
1.69669 0.21875
1.3316 0.34375
1.63124 0.1875
1.42307 0.25
1.64517 0.21875
1.61193 0.3125
1.49497 0.25
1.47071 0.25
1.50245 0.3125
1.52433 0.21875
1.70086 0.125
1.73263 0.15625
1.26361 0.34375
1.34419 0.34375
1.32353 0.3125
1.54944 0.3125
1.37202 0.28125
1.65918 0.125
1.60276 0.25
1.58443 0.21875
1.56121 0.25
1.40061 0.34375
1.46842 0.25
1.71182 0.21875
1.1777 0.46875
1.60169 0.25
1.38368 0.375
1.60218 0.28125
1.16574 0.4375
1.481 0.28125
1.594 0.15625
1.34204 0.40625
1.50749 0.34375
1.31912 0.25
1.47068 0.40625
1.52995 0.28125
1.70902 0.15625
1.58037 0

...

1.46564 0.21875
1.397 0.3125
1.5187 0.25
1.48932 0.3125
1.54312 0.21875
1.35428 0.34375
1.56764 0.28125
1.16796 0.53125
1.63696 0.28125
1.50384 0.25
1.30323 0.40625
1.45355 0.3125
1.38309 0.3125
1.52507 0.1875
1.23873 0.375
1.44094 0.3125
1.66202 0.1875
1.62433 0.25
1.30071 0.21875
1.77955 0.15625
1.56496 0.3125
1.4488 0.3125
1.63113 0.1875
1.28748 0.3125
1.355 0.28125
1.73652 0.125
1.32726 0.34375
1.46371 0.34375
1.46086 0.25
1.52576 0.3125
1.32571 0.375
1.34133 0.4375
1.26197 0.40625
1.4834 0.25
1.45534 0.28125
1.44922 0.34375
1.37442 0.46875
1.7036 0.1875
1.38285 0.40625
1.57353 0.28125
1.69029 0.15625
1.3842 0.34375
1.54435 0.28125
1.59702 0.25
1.76607 0.1875
1.33074 0.34375
1.54265 0.1875
1.20599 0.40625
1.48461 0.28125
1.5916 0.3125
1.71318 0.1875
1.37646 0.28125
1.46714 0.3125
1.34251 0.28125
1.53086 0.25
1.48389 0.3125
1.35936 0.34375
1.14492 0.53125
1.39313 0.375
1.40442 0.34375
1.53831 0.3125
1.5183 0.21875
1.35539 0.375
1.44857 0.28125
1.41112 0.40625
1.49244 0.3125
1.50884 

1.40326 0.3125
1.59648 0.15625
1.48792 0.1875
1.59843 0.28125
1.56717 0.28125
1.48638 0.21875
1.31797 0.375
1.37157 0.3125
1.41036 0.40625
1.37616 0.375
1.6094 0.1875
1.66593 0.15625
1.60472 0.25
1.47006 0.34375
1.32756 0.375
1.61909 0.21875
1.67755 0.25
1.5905 0.21875
1.3749 0.40625
1.68242 0.1875
1.55616 0.1875
1.41327 0.40625
1.48753 0.21875
1.39481 0.3125
1.53264 0.34375
1.48891 0.3125
1.25947 0.46875
1.45055 0.3125
1.26965 0.40625
1.38352 0.34375
1.5974 0.25
1.55264 0.21875
1.40055 0.3125
1.44782 0.34375
1.4322 0.28125
1.58047 0.3125
1.34813 0.34375
1.42121 0.375
1.48919 0.3125
1.34505 0.375
1.50636 0.34375
1.25578 0.375
1.44642 0.3125
1.34552 0.375
1.55665 0.28125
1.56561 0.1875
1.60517 0.21875
1.35435 0.21875
1.56546 0.28125
1.34542 0.4375
1.68471 0.1875
1.50628 0.28125
1.61187 0.25
1.59769 0.21875
1.36202 0.28125
1.24498 0.40625
1.33856 0.40625
1.26027 0.40625
1.26501 0.375
1.70636 0.15625
1.58117 0.21875
1.47538 0.3125
1.43293 0.25
1.26361 0.4375
1.53746 0.125
1.55539 0.21875


1.41457 0.3125
1.56734 0.25
1.51003 0.1875
1.39128 0.40625
1.7255 0.15625
1.53215 0.34375
1.32898 0.375
1.37114 0.4375
1.46105 0.3125
1.43054 0.3125
1.64503 0.28125
1.1268 0.4375
1.78644 0.09375
1.57965 0.15625
1.31968 0.375
1.60466 0.25
1.36915 0.375
1.45117 0.34375
1.36456 0.34375
1.34092 0.375
1.474 0.28125
1.4184 0.25
1.37326 0.40625
1.599 0.25
1.42502 0.28125
1.57292 0.1875
1.68306 0.25
1.68902 0.21875
1.51277 0.28125
1.50526 0.28125
1.24343 0.5
1.55266 0.28125
1.50235 0.21875
1.47049 0.25
1.42633 0.40625
1.55613 0.1875
1.33135 0.34375
1.45229 0.375
1.39818 0.28125
1.65301 0.28125
1.45783 0.375
1.47903 0.3125
1.46929 0.25
1.50592 0.21875
1.44405 0.375
1.47309 0.3125
1.48363 0.25
1.59353 0.1875
1.5646 0.25
1.45119 0.3125
1.39797 0.40625
1.57482 0.1875
1.22842 0.40625
1.35152 0.375
1.41372 0.28125
1.41326 0.34375
1.5864 0.1875
1.36763 0.375
1.31663 0.40625
1.33762 0.3125
1.47309 0.28125
1.53781 0.28125
1.55192 0.375
1.48969 0.21875
1.41014 0.375
1.45919 0.25
1.46376 0.40625
1.46116 

1.53385 0.28125
1.64129 0.3125
1.44023 0.28125
1.60188 0.28125
1.62336 0.25
1.62826 0.21875
1.32076 0.4375
1.5245 0.25
1.44614 0.3125
1.36668 0.375
1.60597 0.21875
1.41164 0.375
1.51362 0.25
1.4493 0.375


KeyboardInterrupt: 